In [1]:
##metaheuristique
import numpy as np
import requests


In [6]:

url = "https://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/mknap1.txt"

try:
    response = requests.get(url)
    response.raise_for_status()  # Vérifie qu'il n'y a pas d'erreur HTTP
    contenu_texte = response.text
    
    with open("mknap1.txt", "w") as fichier:
            fichier.write(contenu_texte)

except requests.exceptions.RequestException as e:
    print(f"Erreur lors de l'ouverture de l'URL : {e}")

In [5]:
base_url = "https://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/mknapcb"


for i in range(9):
    url = base_url + str(i+1) + ".txt"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie qu'il n'y a pas d'erreur HTTP
        contenu_texte = response.text

        with open("mknapcb" + str(i+1) + ".txt", "w") as fichier:
            fichier.write(contenu_texte)


    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de l'ouverture de l'URL : {e}")
    
    

In [83]:
def extract_data(mon_fichier):
    with open(mon_fichier, "r") as fichier:
        contenu = fichier.read()
        contenu = contenu.split()
        #print(contenu)
        nb_instances = int(contenu[0])
        #print(nb_instances)
        instances = {}
        compteur = 1
        for i in range(1,nb_instances):
            elements = contenu[compteur:]
            n  = int(elements[0])
            #print(n)
            m = int(elements[1])
            #print(m)
            val_opt = float(elements[2])
            cost = np.array([float(elements[e]) for e in range(3,3+n)])
            #print(cost)
            a = np.array([int(elements[e]) for e in range(3+n,3+n+n*m)])
            a = a.reshape(m,n)
            b = np.array([int(elements[e]) for e in range(3+n*m,3+n*m+m)])
            instances[i] = [n,m,val_opt,cost,a,b]
            compteur += 3+n+m*n+m
        return nb_instances, instances

In [93]:
mon_fichier = "C:/Users/Thibaud/3A/metaheuristique/Data/mknapcb1.txt"
nb_instances, instances = extract_data(mon_fichier)

a = instances[1][4]
cost = instances[1][3]
len(cost)


100